In [5]:
sweep_config = {
    'method': 'random',  # or 'grid', 'bayes'
    'metric': {
        'name': 'accuracy',
        'goal': 'maximize'
    },
    'parameters': {
        'learning_rate': {
            'values': [0.01, 0.001, 0.0001]
        },
        'batch_size': {
            'values': [16, 32, 64]
        },
        'epochs': {
            'values': [5, 10, 20]
        }
    }
}


In [6]:
import wandb

# Initialize wandb
wandb.init(project='your-project-name')

# Create sweep
sweep_id = wandb.sweep(sweep_config, project='your-project-name')


Create sweep with ID: ftdoknyt
Sweep URL: https://wandb.ai/vickie02736/your-project-name/sweeps/ftdoknyt


In [7]:
import wandb
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Sample dataset
X_train = torch.randn(100, 10)
y_train = torch.randint(0, 2, (100,))

# Define a simple model
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(10, 50)
        self.fc2 = nn.Linear(50, 2)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

def train(config=None):
    with wandb.init(config=config):
        config = wandb.config

        # Set up data loader with batch size from config
        train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=config.batch_size, shuffle=True)
        
        # Initialize the model, criterion, and optimizer with learning rate from config
        model = SimpleNN()
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=config.learning_rate)
        
        # Training loop
        for epoch in range(config.epochs):
            model.train()
            for batch in train_loader:
                inputs, labels = batch
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
            
            # Evaluate model and log metrics
            correct = 0
            total = 0
            model.eval()
            with torch.no_grad():
                for batch in train_loader:
                    inputs, labels = batch
                    outputs = model(inputs)
                    _, predicted = torch.max(outputs.data, 1)
                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()
            accuracy = correct / total
            wandb.log({'accuracy': accuracy, 'loss': loss.item()})

# Step 5: Run the Sweep
wandb.agent(sweep_id, function=train, count=10)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


<IPython.core.display.HTML object>
<IPython.core.display.HTML object>
